In [3]:
import geopandas as gpd
import rioxarray
import rasterio
import pandas as pd
from rasterio.features import rasterize
import numpy as np

In [4]:
# Load the GeoPackage using geopandas
gdf = gpd.read_file('PDSI_Jabar_11_24.gpkg')
gdf.head(2)

,ID,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,2011-07,2011-08,2011-09,...,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,geometry
0,ID_1,"-253,606383","-294,0423729","-265,0367833","-106,7803823","179,6556076","161,3168049","144,3168049","129,3168049","116,3168049",...,,,,,,,,,,"POLYGON ((108.47500 -7.77500, 108.52500 -7.775..."
1,ID_2,"-266,4593694","-310,126506","-276,642143","-116,2028967","176,9506537","158,5403743","142,2922328","127,7025122","114,7025122",...,,,,,,,,,,"POLYGON ((108.42500 -7.77500, 108.47500 -7.775..."


In [6]:
U=gdf.columns.tolist()
for i in ['ID','2024-08','2024-09','2024-10','2024-11','2024-12']:
    U.remove(i)

In [7]:
# Ensure the attribute columns are numeric
def convert(x):
    try:
        return float(x.strip().replace(',','.'))
    except:
        return np.nan
for index, rows in gdf.iterrows():
    for attr in U:
        if attr!='geometry':
            gdf.at[index,attr]=convert(rows[attr])

In [8]:
# Open the reference TIFF with rioxarray to get its properties
reference_raster = rioxarray.open_rasterio('Curah_Hujan.tif')

# Get the spatial properties from the reference raster
transform = reference_raster.rio.transform()
width, height = reference_raster.rio.width, reference_raster.rio.height
crs = reference_raster.rio.crs

# Prepare the shapes for rasterization
raster_bands = []
for attr in U:
    if attr!='geometry':
        shapes = [(geom, value) for geom, value in zip(gdf.geometry, gdf[attr])]
        raster = rasterize(shapes, out_shape=(height, width), transform=transform)
        raster_bands.append(raster)

# Create a new multi-band raster dataset
with rasterio.open(
    'PDSI_11_24_full.tif',
    'w',
    driver='GTiff',
    height=height,
    width=width,
    count=len(U)-1,
    dtype=rasterio.float32,
    crs=crs,
    transform=transform,
) as dst:
    for idx, band in enumerate(raster_bands, start=1):
        dst.write(band.astype(rasterio.float32), idx)